In [1]:
!rm  -f ~/.letta/sqlite.db

In [2]:
import letta
from letta.schemas.memory import ChatMemory

import chromadb

from langchain_openai import OpenAIEmbeddings
from langchain.vectorstores import Chroma

import os
import sys
super_directory = os.path.abspath('..')
sys.path.append(super_directory)

from modules.prompts import sanjay_persona, system_prompt

In [3]:
# Creating the letta_client and setting default LLM and Embedding Model
letta_client = letta.create_client()

letta_client.set_default_llm_config(letta.LLMConfig.default_config("gpt-4o-mini")) 
letta_client.set_default_embedding_config(letta.EmbeddingConfig.default_config("text-embedding-ada-002")) 

In [4]:
# Setting agent name
agent_name = 'sanjay_sarma'

In [5]:
# Getting the vector_db with Sanjay Sarma's information
chroma_client = chromadb.PersistentClient(path='../chromadb')
sanjay_collection = chroma_client.get_collection(name='sanjay_sarma')

vdb_embed_model = OpenAIEmbeddings(model='text-embedding-ada-002')

vectordb = Chroma(
    collection_name='sanjay_sarma',
    embedding_function=vdb_embed_model,
    persist_directory="../chromadb")

/tmp/ipykernel_688050/2753136756.py:7: LangChainDeprecationWarning: The class `Chroma` was deprecated in LangChain 0.2.9 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-chroma package and should be used instead. To use it run `pip install -U :class:`~langchain-chroma` and import as `from :class:`~langchain_chroma import Chroma``.
  vectordb = Chroma(


In [6]:
# Function as tool
def get_sanjay_information(query: str):
    """
    Retrieves relevant information related to a query.

    Args:
        query (str): The query string for which information is being retrieved.

    Returns:
        str: The concatenated content of the relevant search results based on the query.
    """
    from langchain_openai import OpenAIEmbeddings
    from langchain.vectorstores import Chroma
    
    vdb_embed_model = OpenAIEmbeddings(model='text-embedding-ada-002')
    vectordb = Chroma(
        collection_name='sanjay_sarma',
        embedding_function=vdb_embed_model,
        persist_directory="../chromadb")
    
    text = ""
    results = vectordb.similarity_search(query, k=5)
    for result in results:
        text += result.page_content
    return text

get_info_tool = letta_client.create_tool(get_sanjay_information)

In [7]:
# Create sanjay agent
agent_state = letta_client.create_agent(
    name=agent_name,
    tool_ids=[get_info_tool.id],
    memory=ChatMemory(
        human="",
        persona=sanjay_persona
    )
)

print(f"Created agent with name {agent_state.name} and unique ID {agent_state.id}")

Created agent with name sanjay_sarma and unique ID agent-bc835e0b-5fdf-4783-a46b-da2f17de0898


In [8]:
# Updating the system prompt
agent_state.system = system_prompt

In [9]:
# Getting and printing response
user_message = "which story are you referring to that inspired your education"

response = letta_client.send_message(
    agent_id=agent_state.id, 
    message=user_message, 
    role="user" 
)
response

LettaResponse(messages=[ReasoningMessage(id='message-f0f7c66e-d192-49a0-884b-48871ba628fc', date=datetime.datetime(2025, 1, 22, 17, 45, 19, 289841, tzinfo=datetime.timezone.utc), message_type='reasoning_message', reasoning="The user's question prompts a reflection on my educational inspirations. I should share a personal anecdote or significant story that influenced my journey. Gathering information helps clarify what story I should highlight."), ToolCallMessage(id='message-f0f7c66e-d192-49a0-884b-48871ba628fc', date=datetime.datetime(2025, 1, 22, 17, 45, 19, 289841, tzinfo=datetime.timezone.utc), message_type='tool_call_message', tool_call=ToolCall(name='get_sanjay_information', arguments='{\n  "query": "which story inspired your education",\n  "request_heartbeat": true\n}', tool_call_id='call_Zjqhnx1zeGiHEZqnhjxu8Ere')), ToolReturnMessage(id='message-1554447d-cb9a-4b1c-bc3b-89b5c932106d', date=datetime.datetime(2025, 1, 22, 17, 45, 20, 147876, tzinfo=datetime.timezone.utc), message_type='tool_return_message', tool_return='{\n  "status": "OK",\n  "message": "Sanjay Sarma: I was constantly surrounded by knowledge and curiosity. It was truly a blessing.\\"7. Curiosity and passion for learning: Although not explicitly linked to his Indian  education, Sarma\'s childhood in India is described as shaping \\"his curiosity and passion  for learning,\\" which has been a driving force in his career. \\n\\nBased on the interview excerpts, several factors inspired Sanjay Sarma to pursue a  career in academia and innovation:Sanjay Sarma: Education is similar to organizing a library. If you toss a bunch of books into a pile, it’s overwhelming and unhelpful. But if you organize the books into categories—fluid mechanics, philosophy, or anything else—and understand how they connect, it’s transformative. Context creates the \'slots\' in your mind where knowledge can settle and be retrieved when needed.Sanjay Sarma: \\"When I was 14, my grandfather was in the military. He actually served in Malaysia,  during World War II. When I was around 13 or 14, my mother and aunt sat my cousin and me down for a serious conversation. They said, \'We’re educated, but we’re not rich. If you get into IIT, life will be easier for you.\' That advice stayed with me, and I began studying hard for the IIT entrance exams. Eventually, I got in, which was a defining moment in my life. IIT wasn’t just a place where I gainedSanjay Sarma: \\"I consider myself very lucky and privileged to have grown up in India. I come from an upper-caste family with highly educated parents. My father was a senior civil servant and a physicist, and my mother, a teacher, had a deep love for history. Conversations in my family often revolved around history, math, physics, politics, and Gandhi. My grandparents and other relatives were also incredibly well-read. Growing up in such a stimulating environment, I was constantly surrounded by knowledge and",\n  "time": "2025-01-22 05:45:20 PM UTC+0000"\n}', status='success', tool_call_id='call_Zjqhnx1zeGiHEZqnhjxu8Ere', stdout=None, stderr=None), ReasoningMessage(id='message-dd9a68b5-eae5-49e7-91be-78e2a4cf4e0d', date=datetime.datetime(2025, 1, 22, 17, 45, 23, 575332, tzinfo=datetime.timezone.utc), message_type='reasoning_message', reasoning='I have a powerful story to share about the formative experiences in my education. I can articulate how my upbringing and influences led me to pursue a career in academia and innovation.'), ToolCallMessage(id='message-dd9a68b5-eae5-49e7-91be-78e2a4cf4e0d', date=datetime.datetime(2025, 1, 22, 17, 45, 23, 575332, tzinfo=datetime.timezone.utc), message_type='tool_call_message', tool_call=ToolCall(name='send_message', arguments='{\n  "message": "The story that profoundly inspired my education stems from my childhood in India. When I was around 14, my mother and aunt emphasized the importance of education, particularly telling me that if I aimed for IIT (Indian Institute of Technology), life would become easier. Th

In [11]:
# import json

# for message in response.dict()['messages']:
#     if message.get('message_type') == 'tool_call_message' and message.get('tool_call', {}).get('name') == 'send_message':
#         message_reply = message['tool_call']['arguments']
        
#         print(json.loads(message_reply)['message'])